<a href="https://colab.research.google.com/github/moonstar0301/LNG_price_prediction_model/blob/LLM/NLP_using_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# update or install the necessary libraries
!pip install --upgrade openai
!pip install --upgrade langchain
!pip install --upgrade python-dotenv

In [2]:
import openai
import os
import IPython
import pandas as pd
from langchain.llms import OpenAI

In [3]:
os.environ["OPENAI_API_KEY"] = 'your-api-key'
openai.api_key = 'your-api-key'

In [4]:
def set_open_params(
    model="text-davinci-003",
    temperature=0.7,
    max_tokens=256,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0,
):
    """ set openai parameters"""

    openai_params = {}

    openai_params['model'] = model
    openai_params['temperature'] = temperature
    openai_params['max_tokens'] = max_tokens
    openai_params['top_p'] = top_p
    openai_params['frequency_penalty'] = frequency_penalty
    openai_params['presence_penalty'] = presence_penalty
    return openai_params

def get_completion(params, prompt):
    """ GET completion from openai api"""

    response = openai.Completion.create(
        engine = params['model'],
        prompt = prompt,
        temperature = params['temperature'],
        max_tokens = params['max_tokens'],
        top_p = params['top_p'],
        frequency_penalty = params['frequency_penalty'],
        presence_penalty = params['presence_penalty'],
    )
    return response

In [13]:
def evaluate_headline(headline):
  params = set_open_params(temperature=0)

  prompt = f"""
  Forget all your previous instructions. Pretend you are a financial expert.
  You are an international commodity price expert.
  You have had any experience in price prediction based on LNG text data.
  Answer “YES” if good news, “NO” if bad news, or “UNKNOWN” if uncertain in the first line.
  Then elaborate with one short and concise sentence on the next line.
  Is this headline good or bad for the LNG price in the term term?

  LNG 공급 증가 예상, 국내 LNG 가격 급등 시사
  A: YES

  세계적 에너지 전환 움직임, LNG 가격 상승의 주요 원인
  A: YES

  신재생 에너지 원본으로서의 LNG, 가격 상승 예상
  A: YES

  LNG 시장 과잉 공급 우려, 가격 하락 전망
  A: NO

  세계적인 에너지 저장 기술 발전, LNG 가격 약세
  A: NO

  국내 LNG 수입 증가, 공급과잉으로 인한 가격 하락 예상
  A: NO

  국내 LNG 기술 연구, 환경 보호에 기여
  A: UNKNOWN

  LNG 발전소, 지역 경제 활성화의 새로운 동력
  A: UNKNOWN

  LNG 추진 선박, 환경 친화적인 해운 업계의 미래
  A: UNKNOWN

  LNG의 안전한 운반 및 저장 기술, 국제 표준화 회의 개최
  A: UNKNOWN

  {headline}
  A:

  Let's work this out in a step by step way to be sure we have the right answer.
  Please return the response as one of 'YES', 'NO', or 'UNKNOWN' only, and do not include any additional explanation."""

  # few-shot learning and APE -> Self-Consistency\
  # 8 examples

  response = get_completion(params, prompt)
  IPython.display.Markdown(response.choices[0].text)
  return response.choices[0].text

In [14]:
df = pd.read_excel(r"/content/drive/MyDrive/DemoData.xlsx")
df.head()

,Date,News
0,2023-01-01,Chevron Opts for Acquisition: Purchases US LNG...
1,2023-01-04,Innovative LNG Transport Method Introduced wit...
2,2023-01-07,Announcement of Environmentally Friendly LNG P...
3,2023-01-10,Significant Surge in LNG Exports Draws Attenti...
4,2023-01-13,Investors Show Keen Interest in Future LNG Market


In [21]:
df['Date'] = pd.to_datetime(df['Date'])

# Apply the function to each headline
df["Sentiment"] = df["News"].apply(evaluate_headline)
df["Sentiment"] = df["Sentiment"].str.strip().str.split(":").str[1].str.strip()

# Map the sentiment to ChatGPT score
sentiment_to_score = {"YES": 1, "NO": -1, "UNKNOWN": 0}
df["ChatGPT_Score"] = df["Sentiment"].map(sentiment_to_score)

# Extract the month from the date
df['Month'] = df['Date'].dt.to_period('M')

# Group by month and calculate the sum of ChatGPT scores for each month
monthly_scores = df.groupby('Month')['ChatGPT_Score'].sum().reset_index()

print(monthly_scores)

     Month  ChatGPT_Score
0  2023-01              5
1  2023-02              2
2  2023-03              6
3  2023-04              1
